In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
s="‪C:\\Users\\anirudh\\Downloads\\datasets_play_tennis.csv"
s = s.lstrip('\u202a')

In [3]:
data=pd.read_csv(s)

In [4]:
data=data.drop('day',axis=1)

In [5]:
data

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


## Train

In [6]:
def entropy(att1): #Functiion to calculate entropy
    sum1=0
    for i in range(len(att1)):
        pi=list(att1.values())[i]/sum(att1.values())
        sum1+=-pi*math.log2(pi)
    return sum1

In [7]:
def classes(att1): # Entropy for all the columns
    l1=list(dict(data[att1].value_counts()).keys())
    entvalues=dict()
    for i in l1:
        entvalues.update({i:entropy(dict(data[data[att1]==i]['play'].value_counts()))})
    return entvalues    

In [8]:
def gain(S,att1):  #Information gain
    count=dict(data[att1].value_counts())
    entval=classes(att1)
    if entval.keys()==count.keys():
        value=list(entval.values())
        c=list(count.values())
        sum1=0
        for i in range(len(value)):
            sum1+=(c[i]/sum(c))*value[i]
        return (S-sum1)

In [9]:
def root(S,col): # Decides the root node
    result={}
    for i in col:
        result.update({i:gain(S,i)})
    return result   

In [10]:
S=entropy(dict(data['play'].value_counts()))

In [12]:
res=max(root(S,list(data.columns)[:-1]),key=root(S,list(data.columns)[:-1]).get)

In [13]:
res

'outlook'

In [14]:
f={0:res}

In [19]:
def split(att1,l1,i): # Splits the attribute
    count=dict(att1['play'].value_counts())
    if len(count)>1:
        S=entropy(count)
        if len(l1)==len(list(data.columns)):
            left=l1
        else:
            left=list(data.columns.delete(data.columns.get_indexer_for(l1)))
        out=max(root(S,left),key=root(S,left).get)
    else:
        out=list(count.keys())[0]
    return out

In [20]:
def tree(attr,l1): # Tree
    l2=l1
    level={}
    for (i,j) in attr.items():
            if j != "Yes" and j!="No":
                if len(attr)<2:
                    j=res
                elif len(attr)>2:
                    j=j
                for k in list(dict(data[j].value_counts()).keys()):
                    if len(attr)<2:
                        second=data[data[j]==k]
                    else:
                        for z in range(len(l2)):
                            if i[0] in data[l2[z]].unique() or i[1] in data[l2[z]].unique():
                                second=data[((data[l2[z]]==i[0]) | (data[l2[z]]==i[1])) & (data[j]==k)]
                    l1.append(j)
                    l1=list(np.unique(l1))
                    result=split(second,l1,k)
                    if len(attr)<2:
                        level.update({(j,k):result})
                    else:
                        level.update({(i[1],k):result})
    return level,l1
               

In [21]:
m2={}
def store(attr,node): # recusrive function throughout the data
    if attr=={}:
        return m2,node
    else:
        for (i,j) in attr.items():
            if (j == "Yes" or j=="No"):
                m2.update({i:j})
        attr,l=tree(attr,node)
        return store(attr,l)   

In [22]:
attr1,lm=store(f,['play'])
print(lm,attr1)

['humidity', 'outlook', 'play', 'wind'] {('outlook', 'Overcast'): 'Yes', ('Sunny', 'High'): 'No', ('Sunny', 'Normal'): 'Yes', ('High', 'Weak'): 'Yes', ('High', 'Strong'): 'No', ('Normal', 'Weak'): 'Yes', ('Normal', 'Strong'): 'No'}


## Test

In [26]:
def test1(attr,test):
    test.pop(int(data.columns.get_indexer([lm[0]])))
    if list(attr.keys())[0][1] == test[0]:
        return (list(attr.values())[0])
    else:
        for (i,j) in attr.items():
            for k in range(len(test)):
                if ((test[k],test[k-1]) ==i or (test[k-1],test[k])) ==i:
                    return j
    

In [41]:
test=['Overcast','Cool''Normal','Strong']
print(test1(attr1,test))

Yes


In [42]:
test=['Sunny','Mild','High','Weak']
print(test1(attr1,test))

Yes


In [40]:
test=['Rain','Mild','High','Weak']
print(test1(attr1,test))

Yes
